In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [4]:
class FFN(keras.layers.Layer):
  def  __init__(self, d_model, dff):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(dff, activation="relu")
        self.dense2 = tf.keras.layers.Dense(d_model)

def call(self, inputs):
    outputs = dense1(inputs)
    outputs = dense2(inputs)
    return outputs


In [ ]:
class Block(keras.layers.Layer):
    def __init__(self, d_model: int, dff: int = 2048, heads: int = 8):
        super().__init__()

        #parameters
        self.d_model = d_model
        self.dff = dff
        self.heads = heads

        #layers
        self.ffn = FFN(d_model, dff)
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=heads, key_dim=d_model)